In [71]:
##Commencement : importer les packages

import requests 
import pandas as pd
import json   
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import re

### COMMENCEMENT DU PROGRAMME : Webscrapping sur *AUTOSCOUT24*
importer les packages 

In [72]:
website = "https://www.autoscout24.fr/lst/mercedes-benz?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=F&search_id=cw4927j1c&page=1"

In [73]:
response = requests.get(website)

In [74]:
response.status_code

200

In [75]:
#soup object

soup = BeautifulSoup(response.content,"html.parser")

In [76]:
#resultat
 
results = soup.find_all("article",{"class" : "cldt-summary-full-item listing-impressions-tracking list-page-item ListItem_article__ppamD" })

In [77]:
len(results)

20

TARGER NECESSARY 

- modele 
- prix du vehicule 
- annee de sortie 
- CO2
- info sur seller 
- nombre de revu 

### Generer une boucle et cibler les balises et classes du code source 

In [78]:
name = []
prix = []
info = []
sell = []

for i in range(1,30):
    
    # website dans une variable
    website = "https://www.autoscout24.fr/lst/mercedes-benz?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=F&search_id=cw4927j1c&page="+str(i)

    # resquest au website
    response = requests.get(website)
    
    # soup object
    soup = BeautifulSoup(response.content,"html.parser")
    
    #result
    results = soup.find_all("article",{"class" : "cldt-summary-full-item listing-impressions-tracking list-page-item ListItem_article__ppamD"})
    
    
    for result in results :
        try:
            prix.append(result.find("p",{"class":"Price_price__WZayw"}).get_text())
        except:
            prix.append('n/a')
        try:
            name.append(result.find("h2").get_text())
        except:
            name.append('n/a')  
        try:
            info.append(result.find("div",{"class":"VehicleDetailTable_container__mUUbY"}).get_text())
        except:
            info.append('n/a')  
        try:
            sell.append(result.find("span",{"class":"SellerInfo_name__yjUE6"}).get_text())
        except:
            sell.append('n/a')



#### Nettoyage de données : 

In [89]:
car_dealer = pd.DataFrame({"Modele":name , "prix_en_euro":prix, "info vehicule":info, "Info vendeur" : sell })

In [90]:
## récupérer le kilométrage
car_dealer["Nombre kilometre"] = car_dealer["info vehicule"].str.split("km", expand = True)[0]

In [91]:
car_dealer["Taux_CO2"] = car_dealer["info vehicule"].str.split("km", expand = True)[2]

In [92]:
car_dealer["annee"] = car_dealer["info vehicule"].str.split('km', expand= True)[1]

In [93]:
def first_7_chars(string):
    return string[:7]

In [94]:
## récupérer l'année de sortie du véhicule
car_dealer["annee_sortie"]=car_dealer["annee"].apply(first_7_chars)

In [95]:
def caract_entre(between):
    return between[7:22]

In [96]:
## récupérer la puissance en chevaux
car_dealer["puissance"] = car_dealer["annee"].apply(caract_entre)

In [97]:
def delete_first_30_chars(string):
    return string[20:]

In [98]:
car_dealer["annee"] = car_dealer["annee"].apply(delete_first_30_chars)

In [99]:
def delete_first_10_chars(string):
    return string[10:]

In [100]:
car_dealer["annee"] = car_dealer["annee"].apply(delete_first_10_chars)

In [101]:
def delete_2_last_chars(string):
    return string[:1]

In [102]:
### fonction récupérant uniquement les chiffres
def keep_digits(string):
    digits = ''.join(filter(str.isdigit, string))
    return pd.to_numeric(digits)

In [103]:
car_dealer['Taux_CO2'] = car_dealer['Taux_CO2'].apply(keep_digits)

In [104]:
car_dealer['prix_en_euro'] = car_dealer['prix_en_euro'].apply(keep_digits)

In [105]:
def extract_words(string, words_list):
    words = []
    for word in words_list:
        if word in string:
            words.append(word)
    return words

In [106]:
## récupérer la "transmission" se trouvant dans la colonne "année"
car_dealer['Transmission'] = car_dealer['annee'].apply(lambda x: extract_words(x, ["Semi-automatique","automatique", "manuelle"]))

In [107]:
## récupérer la "consommation" se trouvant dans la colonne "année"

car_dealer['Consommation'] = car_dealer['annee'].apply(lambda x: extract_words(x, ["Diesel","Essence", "Électrique/Essence","Électrique"]))

In [108]:
del car_dealer["annee"]

In [109]:
del car_dealer["info vehicule"]

In [110]:
## utilisation de regex pour extraire les informations d'une parenthese 
car_dealer['puissance'] = car_dealer['puissance'].str.extract(r"\((.*)\)")

In [111]:
car_dealer

,Modele,prix_en_euro,Info vendeur,Nombre kilometre,Taux_CO2,annee_sortie,puissance,Transmission,Consommation
0,Mercedes-Benz AMG GT,109990.0,SASU,45 000,224.0,04/2016,510 CH,[automatique],[Essence]
1,Mercedes-Benz CLK,15990.0,Agence Automobilière de Senlis(2),85 000,276.0,10/2002,306 CH,[automatique],[]
2,Mercedes-Benz G 63 AMG,899500.0,PARIS PRESTIGE CARS(9),315,NaN,12/2021,799 CH,[automatique],[Essence]
3,Mercedes-Benz S 320,14990.0,PACK AUTOS,47 000,266.0,08/2002,223 CH,[automatique],[Essence]
4,Mercedes-Benz CL,36490.0,MERCEDES SAINT MAXIMIN - AUTOSPHERE(3),7 000,NaN,01/2023,- CH,[automatique],[Diesel]
...,...,...,...,...,...,...,...,...,...
395,Mercedes-Benz A 180,26990.0,sasu cg canteleu(1),64 500,NaN,02/2020,116 CH,[automatique],[Diesel]
396,Mercedes-Benz CLA 220,NaN,CapCar(9),113 165,182.0,08/2009,170 CH,[automatique],[Diesel]
397,Mercedes-Benz Vito,59990.0,RENAULT THIONVILLE - AUTOSPHERE(2),10,NaN,08/2022,- CH,[automatique],[Diesel]
398,Mercedes-Benz Vito,36990.0,MERCEDES AMIENS - AUTOSPHERE,89 785,NaN,07/2019,- CH,[automatique],[Diesel]


La base de données étant propre, nous allons faire les derniers ajustement via Microsoft Excel.
L'étape suivante sera de basculer sur un logiciel pour créer des visualisation. Exemple: Tableau

In [77]:
# exportation sous excel
#car_dealer.to_excel("autoscout_excel_2.xlsx", index= False)
#car_dealer.to_csv('car_dealer.csv',encoding='utf-8')